### Set path to original pyNeuroChem. Please change to your own path

#### Import required libraries

In [1]:
import sys
#sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANIENS
from ase_interface import aniensloader

van Der Waals correction will be unavailable. Please install ased3


In [2]:
import numpy as np
import  ase
import time
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS 

#### Load the benzene molecule from the xyz

In [3]:
bz = read('data/bz.xyz')

#### Set ANI ensemble calculator

In [4]:
# Current ANI model options are:
# '../ani_models/ani-1ccx_8x.info' Coupled cluster transfer learned model
# '../ani_models/ani-1x_8x.info'   Full ANI-1x wb97x/6-31g* dataset model
bz.set_calculator(ANIENS(aniensloader('../ani_models/ani-1ccx_8x.info',0,multigpu=False)))

#### Run Optimization

In [5]:
start_time = time.time()
dyn = LBFGS(bz)
dyn.run(fmax=0.001)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

       Step     Time          Energy         fmax
LBFGS:    0 15:54:49    -6311.279512        0.5988
LBFGS:    1 15:54:49    -6311.289039        0.2507
LBFGS:    2 15:54:49    -6311.292550        0.1099
LBFGS:    3 15:54:49    -6311.293796        0.0610
LBFGS:    4 15:54:49    -6311.294387        0.0328
LBFGS:    5 15:54:49    -6311.294429        0.0054
LBFGS:    6 15:54:49    -6311.294432        0.0003
[ANI Total time: 0.06440186500549316 seconds]


#### Setup dynamics

In [6]:
# Temperature
T = 300.0

# We want to run MD with constant energy using the Langevin algorithm
# with a time step of 5 fs, the temperature T and the friction
# coefficient to 0.02 atomic units.
dyn = Langevin(bz, 0.25 * units.fs, T * units.kB, 0.02)

#### Setup printer function

In [7]:
def printenergy(a=bz):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
dyn.attach(printenergy, interval=100)
printenergy()

Energy per atom: Epot = -525.941eV  Ekin = 0.000eV (T=  0K)  Etot = -525.941eV


#### Run 2000 steps of MD

In [8]:
start_time = time.time()
dyn.run(2000) # Do 1ps of MD
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Energy per atom: Epot = -525.940eV  Ekin = 0.001eV (T= 10K)  Etot = -525.939eV
Energy per atom: Epot = -525.938eV  Ekin = 0.003eV (T= 24K)  Etot = -525.935eV
Energy per atom: Epot = -525.937eV  Ekin = 0.007eV (T= 51K)  Etot = -525.931eV
Energy per atom: Epot = -525.934eV  Ekin = 0.005eV (T= 38K)  Etot = -525.929eV
Energy per atom: Epot = -525.934eV  Ekin = 0.006eV (T= 50K)  Etot = -525.928eV
Energy per atom: Epot = -525.933eV  Ekin = 0.009eV (T= 70K)  Etot = -525.924eV
Energy per atom: Epot = -525.931eV  Ekin = 0.009eV (T= 67K)  Etot = -525.923eV
Energy per atom: Epot = -525.934eV  Ekin = 0.008eV (T= 65K)  Etot = -525.925eV
Energy per atom: Epot = -525.931eV  Ekin = 0.008eV (T= 58K)  Etot = -525.923eV
Energy per atom: Epot = -525.933eV  Ekin = 0.014eV (T=105K)  Etot = -525.920eV
Energy per atom: Epot = -525.933eV  Ekin = 0.013eV (T=103K)  Etot = -525.920eV
Energy per atom: Epot = -525.933eV  Ekin = 0.014eV (T=109K)  Etot = -525.919eV
Energy per atom: Epot = -525.932eV  Ekin = 0.014eV (